In [142]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

nt_to_idx = {'A': 0, 'U': 1, 'G': 2, 'C': 3, 'N': 4}


In [ ]:
class RNAStruct(Dataset):
    def __init__(self, seq_csv, labels_csv):
        seq_df = pd.read_csv(seq_csv)
        seq_map = {row['target_id']: row['sequence'].strip() for _, row in seq_df.iterrows()}

        labels_df = pd.read_csv(labels_csv)
        self.data = {}
        for _, row in labels_df.iterrows():
            target_id = row['ID'].rsplit('_', 1)[0]
            idx = int(row['resid']) - 1
            if target_id not in self.data:
                self.data[target_id] = []
            self.data[target_id].append((idx, row['resname'], row['x_1'], row['y_1'], row['z_1']))

        self.samples = []
        all_coords = []
        for target_id in self.data:
            dat = sorted(self.data[target_id], key=lambda x: x[0])
            seq = seq_map[target_id]
            coords = np.array([[x[2], x[3], x[4]] for x in dat], dtype=np.float32)
            if len(seq) == len(coords) and np.isfinite(coords).all():
                seq_idx = np.array([nt_to_idx.get(nt, 4) for nt in seq], dtype=np.int64)
                self.samples.append((target_id, seq_idx, coords))
                all_coords.append(coords)
            else:
                print(f"Bad entry removed: {target_id} (len/coords/finite)")

        all_coords = np.concatenate(all_coords, axis=0)
        self.mean = np.mean(all_coords, axis=0)
        self.std = np.std(all_coords, axis=0)

    def __len__(self):
        return len(self.samples)
    def __getitem__(self, idx):
        target_id, seq_idx, coords = self.samples[idx]
        norm_coords = (coords - self.mean) / self.std
        return torch.LongTensor(seq_idx), torch.tensor(norm_coords), len(seq_idx), target_id


In [144]:
#collate function
def rna_collate(batch):
    seqs, coords, lengths, ids = zip(*batch)
    lengths = torch.tensor(lengths)
    seqs_padded = pad_sequence(seqs, batch_first=True, padding_value=4)
    coords_padded = pad_sequence(coords, batch_first=True, padding_value=0)
    return seqs_padded, coords_padded, lengths, ids


In [ ]:
# Model
class RNA3DNet(nn.Module):
    def __init__(self, vocab_size=5, emb_dim=128, num_layers=6, nhead=8, ff_dim=256, dropout=0.2):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, emb_dim, padding_idx=4)
        encoder_layer = nn.TransformerEncoderLayer(d_model=emb_dim, nhead=nhead, dim_feedforward=ff_dim, dropout=dropout, batch_first=True)
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.ln = nn.LayerNorm(emb_dim)
        self.fc = nn.Linear(emb_dim, 3)
    def forward(self, seq, lengths):
        x = self.embed(seq)
        mask = (seq == 4)
        x = self.encoder(x, src_key_padding_mask=mask)
        x = self.ln(x)
        coords = self.fc(x)
        return coords

In [146]:
def train_model(model, dataloader, device, epochs=30, lr=1e-3, val_loader=None):
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.8)
    loss_fn = nn.MSELoss(reduction='none')
    best_val_rmsd = float('inf')

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        total_points = 0
        for seqs, coords, lengths, _ in dataloader:
            seqs, coords, lengths = seqs.to(device), coords.to(device), lengths.to(device)
            optimizer.zero_grad()
            pred_coords = model(seqs, lengths)
            mask = torch.arange(seqs.size(1), device=device)[None, :] < lengths[:, None]
            mse = loss_fn(pred_coords, coords).sum(2)
            loss = (mse * mask).sum() / mask.sum()
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * mask.sum().item()
            total_points += mask.sum().item()

        epoch_loss = running_loss / total_points
        print(f"Epoch {epoch+1} Loss: {epoch_loss:.5f}")

        if val_loader is not None:
            val_rmsd = validate(model, val_loader, device)
            if val_rmsd is not None and val_rmsd < best_val_rmsd:
                best_val_rmsd = val_rmsd
                torch.save(model.state_dict(), 'best_model.pt')
                print(f"New best checkpoint (epoch {epoch+1}, RMSD {val_rmsd:.4f})")

        scheduler.step()

    
    torch.save(model.state_dict(), 'rna3dnet_baseline.pt')
    print("Final model saved as rna3dnet_baseline.pt")


In [147]:
#Submission Function
def submission(model, seq_csv, submission_csv, device, mean, std):
    seq_df = pd.read_csv(seq_csv)
    model.eval()
    submissions = []
    for idx, row in seq_df.iterrows():
        target_id, sequence = row['target_id'], row['sequence'].strip()
        seq_idx = torch.LongTensor([nt_to_idx.get(nt, 4) for nt in sequence]).unsqueeze(0).to(device)
        lengths = torch.tensor([len(sequence)]).to(device)
        with torch.no_grad():
            coords = model(seq_idx, lengths)[0][:len(sequence)].cpu().numpy()
            coords = coords * std + mean
        for i, (nt, (x, y, z)) in enumerate(zip(sequence, coords)):
            id_out = f"{target_id}_{i+1}"
            submissions.append({
                "ID": id_out,
                "resname": nt,
                "resid": i+1,
                "x_1": x, "y_1": y, "z_1": z,
                "x_2": 0.0, "y_2": 0.0, "z_2": 0.0,
                "x_3": 0.0 
            })
    sub_df = pd.DataFrame(submissions)
    sub_df.to_csv(submission_csv, index=False)
    print(f"Submission CSV written: {submission_csv}")


In [ ]:
#Training 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 32

train_dataset = RNAStruct("rna-folding/train_sequences_clean.csv", "rna-folding/train_labels_clean.csv")
val_dataset = RNAStruct("rna-folding/validation_sequences.csv", "rna-folding/validation_labels.csv")

mean = train_dataset.mean
std = train_dataset.std

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=rna_collate)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=rna_collate)

model = RNA3DNet().to(device)
train_model(model, train_loader, device, epochs=30, lr=5e-4, val_loader=val_loader)

model.load_state_dict(torch.load('rna3dnet_baseline.pt'))

# Generate predictions
submission(model, "rna-folding/test_sequences.csv", "rna-folding/submission.csv", device, mean, std)


Epoch 1 Loss: 3.60013
Validation RMSD: 0.336
Epoch 2 Loss: 3.27215
Validation RMSD: 0.681
Epoch 3 Loss: 3.10564
Validation RMSD: 0.520
Epoch 4 Loss: 3.21520
Validation RMSD: 1.177
Epoch 5 Loss: 3.56804
Validation RMSD: 0.791
Epoch 6 Loss: 3.27372
Validation RMSD: 0.291
Epoch 7 Loss: 3.11177
Validation RMSD: 0.585
Epoch 8 Loss: 3.35639
Validation RMSD: 0.283
Epoch 9 Loss: 3.25554
Validation RMSD: 0.448
Epoch 10 Loss: 3.03878
Validation RMSD: 0.466
Epoch 11 Loss: 3.15604
Validation RMSD: 0.335
Epoch 12 Loss: 3.09856
Validation RMSD: 0.908
Epoch 13 Loss: 3.22699
Validation RMSD: 0.388
Epoch 14 Loss: 3.01949
Validation RMSD: 0.463
Epoch 15 Loss: 2.95320
Validation RMSD: 0.466
Epoch 16 Loss: 3.04773
Validation RMSD: 0.329
Epoch 17 Loss: 3.01918
Validation RMSD: 0.490
Epoch 18 Loss: 3.02901
Validation RMSD: 0.613
Epoch 19 Loss: 3.10907
Validation RMSD: 0.496
Epoch 20 Loss: 3.01591
Validation RMSD: 0.474
Epoch 21 Loss: 2.98322
Validation RMSD: 0.307
Epoch 22 Loss: 2.96907
Validation RMSD: 0.2

FileNotFoundError: [Errno 2] No such file or directory: 'best_model.pt'